In [1]:
import pandas as pd
import os

In [2]:
DIR = 'C:/Users/USER/Desktop/python/competition/CarCrashAnalysisAIContest/data/'
os.chdir(DIR)
os.getcwd()
df = pd.read_csv(DIR+'train.csv')
df

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,7
1,TRAIN_0001,./train/TRAIN_0001.mp4,7
2,TRAIN_0002,./train/TRAIN_0002.mp4,0
3,TRAIN_0003,./train/TRAIN_0003.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
...,...,...,...
2693,TRAIN_2693,./train/TRAIN_2693.mp4,3
2694,TRAIN_2694,./train/TRAIN_2694.mp4,5
2695,TRAIN_2695,./train/TRAIN_2695.mp4,0
2696,TRAIN_2696,./train/TRAIN_2696.mp4,0


In [4]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore') 

In [5]:
CFG = {
    'VIDEO_LENGTH':50, # 10프레임 * 5초
    'IMG_SIZE':128,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':4,
    'SEED':41
}

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [7]:
WORKING_DIR = 'C:/Users/USER/Desktop/python/competition/CarCrashAnalysisAIContest/data'

os.chdir(WORKING_DIR)
os.getcwd()

'C:\\Users\\USER\\Desktop\\python\\competition\\CarCrashAnalysisAIContest\\data'

In [8]:
# TRAIN_CSV_PATH = os.path.join(WORKING_DIR, '/train.csv')
df = pd.read_csv(WORKING_DIR+'/train.csv')
df

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,7
1,TRAIN_0001,./train/TRAIN_0001.mp4,7
2,TRAIN_0002,./train/TRAIN_0002.mp4,0
3,TRAIN_0003,./train/TRAIN_0003.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
...,...,...,...
2693,TRAIN_2693,./train/TRAIN_2693.mp4,3
2694,TRAIN_2694,./train/TRAIN_2694.mp4,5
2695,TRAIN_2695,./train/TRAIN_2695.mp4,0
2696,TRAIN_2696,./train/TRAIN_2696.mp4,0


In [17]:
df_crash = df.copy()
def change(x):
    if x != 0:
        return 1
    else:
        return 0
df_crash['label'] = df_crash['label'].apply(change)
df_crash['label'].value_counts()

0    1783
1     915
Name: label, dtype: int64

In [18]:
train, val, _, _ = train_test_split(df_crash, df_crash['label'], test_size=0.2, random_state=CFG['SEED'])

In [19]:
class CustomDataset(Dataset):
    def __init__(self, video_path_list, label_list):
        self.video_path_list = video_path_list
        self.label_list = label_list
        
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(CFG['VIDEO_LENGTH']):
            _, img = cap.read()
            img = cv2.resize(img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
            img = img / 255.
            frames.append(img)
        return torch.FloatTensor(np.array(frames)).permute(3, 0, 1, 2)

In [20]:
train_dataset = CustomDataset(train['video_path'].values, train['label'].values)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['video_path'].values, val['label'].values)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [21]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(videos)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

In [22]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

In [23]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [25]:
import models_code.ResNet3D
model = models_code.ResNet3D.generate_model(18)
# model = torch.load(WORKING_DIR + '/model.pt')
# model = models.video.r3d_18(pretrained=True)
# num_features = model.fc.in_features
# model.fc = nn.Linear(num_features, 13)
model.eval()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 135/135 [01:27<00:00,  1.55it/s]


Epoch [1], Train Loss : [0.63332] Val Loss : [0.48912] Val F1 : [0.63194]


100%|██████████| 135/135 [01:18<00:00,  1.73it/s]


Epoch [2], Train Loss : [0.40965] Val Loss : [0.36931] Val F1 : [0.85247]


100%|██████████| 135/135 [01:20<00:00,  1.69it/s]


...

Epoch [8], Train Loss : [0.13518] Val Loss : [0.20945] Val F1 : [0.91669]


100%|██████████| 135/135 [01:19<00:00,  1.70it/s]


Epoch [9], Train Loss : [0.11668] Val Loss : [0.16512] Val F1 : [0.92955]


100%|██████████| 135/135 [01:19<00:00,  1.70it/s]

Epoch [10], Train Loss : [0.10382] Val Loss : [0.19391] Val F1 : [0.94383]


In [26]:
torch.save(model, WORKING_DIR+'/model.pt')

In [27]:
test = pd.read_csv(WORKING_DIR+'/test.csv')

In [28]:
test_dataset = CustomDataset(test['video_path'].values, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [29]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [30]:
preds = inference(model, test_loader, device)

100%|██████████| 450/450 [04:53<00:00,  1.53it/s]


In [37]:
submit = pd.read_csv(WORKING_DIR+'/sample_submission.csv')

In [38]:
submit['label_crash'] = preds
submit.head()

,sample_id,label,label_crash
0,TEST_0000,0,0
1,TEST_0001,0,0
2,TEST_0002,0,0
3,TEST_0003,0,0
4,TEST_0004,0,0


In [39]:
submit['label_crash'].value_counts()

0    1245
1     555
Name: label_crash, dtype: int64

In [40]:
submit.to_csv('./resnet&ViT_submit.csv', index=False)

In [47]:
df_crash_ex = df.copy()

df_crash_ex = df_crash_ex[df_crash['label']>0]
df_crash_ex

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,7
1,TRAIN_0001,./train/TRAIN_0001.mp4,7
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
6,TRAIN_0006,./train/TRAIN_0006.mp4,3
7,TRAIN_0007,./train/TRAIN_0007.mp4,7
...,...,...,...
2685,TRAIN_2685,./train/TRAIN_2685.mp4,8
2689,TRAIN_2689,./train/TRAIN_2689.mp4,1
2692,TRAIN_2692,./train/TRAIN_2692.mp4,7
2693,TRAIN_2693,./train/TRAIN_2693.mp4,3


In [49]:
df_crash_ex['label'].value_counts()

1     318
7     317
3      78
2      51
9      34
11     33
8      30
5      28
4      13
12      6
10      4
6       3
Name: label, dtype: int64

In [56]:
df_crash_ex_own = df_crash_ex.copy()
def change(x):
    if 0<x<=6:
        return 0
    else:
        return 1
df_crash_ex_own['label'] = df_crash_ex_own['label'].apply(change)
df_crash_ex_own['label'].value_counts()

0    491
1    424
Name: label, dtype: int64

In [64]:
CFG = {
    'VIDEO_LENGTH':50, # 10프레임 * 5초
    'IMG_SIZE':128,
    'EPOCHS':5,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':2,
    'SEED':41
}

In [65]:
train, val, _, _ = train_test_split(df_crash_ex_own, df_crash_ex_own['label'], test_size=0.2, random_state=CFG['SEED'])

In [66]:
train_dataset = CustomDataset(train['video_path'].values, train['label'].values)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['video_path'].values, val['label'].values)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [67]:
import models_code.ResNet3D
model = models_code.ResNet3D.generate_model(18)
# model = torch.load(WORKING_DIR + '/model.pt')
# model = models.video.r3d_18(pretrained=True)
# num_features = model.fc.in_features
# model.fc = nn.Linear(num_features, 13)
# model.eval()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

TypeError: 'DataFrame' object is not callable